In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df.shape

(20800, 5)

In [5]:
df=df.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
x=df.drop('label',axis=1)
y=df['label']

In [8]:
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [10]:
import tensorflow as tf

In [12]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
voc_size=5000

In [61]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/abrar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
def preprocessing_text(x):
    message=x.copy()
    message.reset_index(inplace=True)
    sentences = message['title']
    
    from nltk.stem.porter import PorterStemmer
    ps=PorterStemmer()
    corpus=[]
    for i in range(0, len(message)):
        review=re.sub('[^a-zA-Z]',' ', sentences[i])
        review= review.lower()
        review= review.split()
        
        review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
        review=' '.join(review)
        
        corpus.append(review)
        
    onehot_sent= [one_hot(word,voc_size) for word in corpus]
    padded_sent= pad_sequences(onehot_sent, padding='pre',maxlen=20)


    return padded_sent    
    

In [64]:
padded_sent=preprocessing_text(x)

In [65]:
model=Sequential()
model.add(Embedding(voc_size, 40 , input_length=20))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
import numpy as np

In [68]:
x_final= np.array(padded_sent)
y_final= np.array(y)


In [69]:
x_final.shape

(18285, 20)

In [70]:
y_final.shape

(18285,)

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test= train_test_split(x_final,y_final, test_size=0.3, stratify=y_final, random_state=10)

In [72]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 7s 28ms/step - loss: 0.3198 - accuracy: 0.8523 - val_loss: 0.1817 - val_accuracy: 0.9258
Epoch 2/10
200/200 [==============================] - 3s 17ms/step - loss: 0.1375 - accuracy: 0.9455 - val_loss: 0.1786 - val_accuracy: 0.9244
Epoch 3/10
200/200 [==============================] - 3s 16ms/step - loss: 0.0968 - accuracy: 0.9638 - val_loss: 0.1941 - val_accuracy: 0.9253
Epoch 4/10
200/200 [==============================] - 4s 18ms/step - loss: 0.0690 - accuracy: 0.9762 - val_loss: 0.2183 - val_accuracy: 0.9222
Epoch 5/10
200/200 [==============================] - 4s 18ms/step - loss: 0.0418 - accuracy: 0.9862 - val_loss: 0.2283 - val_accuracy: 0.9147
Epoch 6/10
200/200 [==============================] - 3s 17ms/step - loss: 0.0319 - accuracy: 0.9899 - val_loss: 0.3687 - val_accuracy: 0.9211
Epoch 7/10
200/200 [==============================] - 3s 17ms/step - loss: 0.0178 - accuracy: 0.9946 - val_loss: 0.3599 - val_accuracy: 0.9198

In [73]:
y_pred= model.predict(x_test)

172/172 [==============================] - 1s 3ms/step


In [74]:
y_pred= [round(pred[0]) for pred in y_pred]

In [75]:
y_pred[0]

1

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2840,  269],
       [ 175, 2202]])

In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9190667152752461

In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3109
           1       0.89      0.93      0.91      2377

    accuracy                           0.92      5486
   macro avg       0.92      0.92      0.92      5486
weighted avg       0.92      0.92      0.92      5486


In [79]:
df_test= pd.read_csv('data/test.csv')
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [88]:
df_test.dropna(inplace=True)

In [89]:
padded_sent_test=preprocessing_text(df_test)

In [91]:
padded_sent_test=np.array(padded_sent_test)

In [94]:
predictions=model.predict(padded_sent_test)

143/143 [==============================] - 1s 5ms/step


In [97]:
test_pred= [round(pred[0]) for pred in predictions]

In [98]:
test_pred

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,


In [99]:
df_test['pred_label']=test_pred

In [100]:
df_test.head()

,id,title,author,text,pred_label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",0
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori...",0


In [103]:
df_final=df_test.drop(['author','text'],axis=1)
df_final.to_csv('final_pred.csv',sep=',', index=False)